In [1]:
!pip install llama_index[langchain] -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.5/943.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency reso

In [2]:
!pip install pypdf transformers llama-index sentence-transformers python-dotenv -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.8 MB/s eta 0:00:00


In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 27.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.22-cp310-cp310-manylinux_2_35_x86_64.whl size=7835778 sha256=6762ac639f79edfada50db1513c76d7640a7e63e25316008926b74b0560459bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-ch9pdu5e/wheels/64/7e/c4/11fee2bb4b914968fabb2168c237ab1ade9702cfd2c274c4bd
Successfully built llama-cpp-python


In [4]:
import logging
import sys
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
import torch
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
#from llama_index import LangchainEmbedding

logging.basicConfig(stream=sys.stdout,level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

documents = SimpleDirectoryReader("Data/").load_data()

In [5]:
llm = LlamaCPP(
    model_url='https://huggingface.co/TheBloke/zephyr-7B-alpha-GGUF/resolve/main/zephyr-7b-alpha.Q5_K_M.gguf',
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers":-1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

total size (MB): 5131.41


4894it [00:27, 175.86it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [6]:
embed_model = HuggingFaceEmbeddings(model_name="thenlper/gte-large")


service_context = ServiceContext.from_defaults(chunk_size=256,
                                               llm=llm,
                                               embed_model=embed_model)
index = VectorStoreIndex.from_documents(documents,service_context=service_context)
query_engine = index.as_query_engine()

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
while True:
  query = input()
  response = query_engine.query(query)
  print(response)

What do you mean by 3D object representation?
 <<SYS>>
In the context of computer graphics, 3D object representation refers to the methods used to describe three-dimensional objects in a digital format. These representations can be divided into two broad categories: boundary representations (B-reps) and space-partitioning representations. B-reps describe an object as a set of surfaces that separate its interior from the environment, while space-partitioning representations partition the spatial region containing an object into small, non-overlapping, contiguous solids to describe interior properties. Examples of boundary representations include polygon surfaces and spline patches, while examples of space-partitioning representations include octree representation.
What are different ways of representing 3D graphs in computer?


Llama.generate: prefix-match hit


 <<SYS>>
The given context information is about 3D object representation in computer graphics. The query is not related to this topic. Therefore, there is no answer for the given query based on the provided context information.
How does computer represent 3D graphs?


Llama.generate: prefix-match hit


 <<SYS>>
Computers do not directly represent 3D graphs as they are typically used to represent relationships between data points in a two-dimensional space. However, in the context of 3D object representation, computer graphics software can create and manipulate 3D objects using various techniques such as boundary representations (B-reps) that describe a set of surfaces enclosing the object's interior. These surfaces can be represented using polygons or spline patches.
Tell me about Boundary representations of 3D graphs.


Llama.generate: prefix-match hit


 <<SYS>>
Boundary representations (B-reps) are a category of solid object representation schemes in 3D graphics that describe an object as a set of surfaces that enclose its interior, separating it from the environment. B-reps are commonly used to represent complex objects with high precision, such as polygon and spline patches. In contrast to other representations, B-reps do not describe the object's interior or volume, but rather its exterior. An example of a boundary representation is a wireframe, which consists of lines that connect to create polygons representing three-dimensional shapes. The number of polygons within a wireframe is typically an indicator of how detailed the 3D model is.


KeyboardInterrupt: ignored